# Lab7 - Klasteryzacja

### Przygotowanie danych:

In [1]:
from sklearn.datasets import fetch_openml 
import numpy as np
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)
X = mnist["data"]
y = mnist["target"]

### Klasteryzacja

In [2]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [3]:
kmeans_sil = []
y_pred_forTen = 0

for k in range(8, 13): 
    kmeans = KMeans(n_clusters=k, random_state=42)
    y_pred = kmeans.fit_predict(X)
    if k == 10:
        y_pred_forTen = y_pred
    sil_scr = silhouette_score(X, y_pred)
    kmeans_sil.append(sil_scr)

In [4]:
import pickle
with open('kmeans_sil.pkl','wb') as f: pickle.dump(kmeans_sil, f)

#sprawdzenie zawartosci pliku
with open('kmeans_sil.pkl','rb') as f: print(pickle.load(f))

[0.07338401965925966, 0.05683381826333592, 0.05869201052848778, 0.05836351154790604, 0.05894738457992754]


score powinien być największy dla -> 10 <-

In [5]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y, y_pred_forTen)
print(conf_matrix)

argmax = np.argmax(conf_matrix, axis=1)
print(argmax)

[[ 290    2 1265   39    9    7  162    4   72 5053]
 [   8 4293    7    7   10   11    7 3526    8    0]
 [ 323  423  246  216 4863   78  147  436  201   57]
 [4581  449  461  193  216   45   31   58 1083   24]
 [   0  178  288 3728   29 2173  168  234   17    9]
 [2129  155 1812  432    7  215   67  280 1156   60]
 [  38  190 2068   67   53    4 4326   45   14   71]
 [   6  372   12 2094   53 4399    4  314   18   21]
 [1212  335  292  208   53  193   51  330 4115   36]
 [  87  261   31 3462   19 2849   16   95   87   51]]
[9 1 4 0 3 0 6 5 8 3]


In [6]:
kmeans_argmax = []

for row, index in zip(conf_matrix, argmax):
    kmeans_argmax.append(row[index])

kmeans_argmax = list(set(kmeans_argmax))
kmeans_argmax.sort()

In [7]:
with open('kmeans_argmax.pkl','wb') as f: pickle.dump(kmeans_argmax, f)

#sprawdzenie zawartosci pliku
with open('kmeans_argmax.pkl','rb') as f: print(pickle.load(f))

[2129, 3462, 3728, 4115, 4293, 4326, 4399, 4581, 4863, 5053]


### DBSCAN

In [8]:
dist_long = np.array([])
for i in range(0,301):
    for j in range(i+1,301):
        x1 = X[i]
        x2 = X[j]
        dist_long = np.append(dist_long, np.linalg.norm(x1-x2))

In [9]:
dist = []
for i in np.sort(dist_long)[:10]:
    dist.append(i)

In [10]:
with open('dist.pkl','wb') as f: pickle.dump(dist, f)

#sprawdzenie zawartosci pliku
with open('dist.pkl','rb') as f: print(pickle.load(f))

[419.583126448145, 524.325280717991, 636.2460215985637, 638.289902787127, 638.4520342202694, 653.3475338592776, 669.5827058698575, 703.4898719953259, 723.5613311945298, 739.4200430066796]


In [11]:
s = np.mean(np.sort(dist_long)[:3])

eps -> [ s , ... , s+10%s ]   krok co 4%s

In [12]:
from sklearn.cluster import DBSCAN

dbscan_len = []

def counter(a):
    b = []
    count = 0
    for i in a:
        if not i in b:
            count += 1
            b.append(i)
    return count

for eps in [s, s+.04*s, s+.08*s]:
    dbscan = DBSCAN(eps)
    dbscan.fit(X)
    print(dbscan.labels_)
    dbscan_len.append(counter(dbscan.labels_))

[-1 -1 -1 ... -1 -1 -1]
[-1 -1 -1 ... -1 -1 -1]
[-1 -1 -1 ... -1 -1 -1]


In [13]:
with open('dbscan_len.pkl','wb') as f: pickle.dump(dbscan_len, f)

#sprawdzenie zawartosci pliku
with open('dbscan_len.pkl','rb') as f: print(pickle.load(f))

[10, 12, 5]
